# Identifying Early Help Referrals for Local Authorities with Machine Learning and Bias Mitigation

#Code for Reproducibility Purpose.


# Preprocessing


In [ ]:
# Import Python libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option("display.float_format", "{:.3f}".format)

from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!ls "/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper"
dataset='LCC_Early_Help_20220822.csv'

In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper/LCC_Early_Help_20220822.csv')
df.head()

## Dropping features


In [ ]:
def drop_columns(df_, col_list):
    df = df_.copy()
    return df.drop(col_list, axis=1)

columnlist = ['LOCALITY_DECISION_DATE', 'TERMS_PREVIOUS', 'YEARS_PREVIOUS', 'PBR_ID', 'PBR_FID2', 'AGE_CURRENT', 'EARLY_HELP_INCIDENCE']
df = drop_columns(df, columnlist)

## One-hot encoding Gender


In [ ]:
def one_hot_gender(df):
    """ One hot encoding of the gender column """
    df_ = df.copy()
    male = [1 if i == 'M' else 0 for i in df_['GENDER']]
    female = [1 if i == 'F' else 0 for i in df_['GENDER']]
    other = [1 if i!= 'M' and i!='F' else 0 for i in df_['GENDER']]

    # Adding new columns to the dataframe
    df_['MALE'] = male
    df_['FEMALE'] = female
    df_['OTHER'] = other

    df_ = df_.drop('GENDER', axis = 1)

    return df_


In [ ]:
df = one_hot_gender(df)

## Dataset of individuals who are under 18 years old. 

**From this point on we only analyse the cleaned data after the columns are removed and of individuals who are less than 18 years old.**

In [ ]:
# This will select records whose age is less than the value defined in the agetoselect variable.
agetoselect = 18 
df = df[df['AGE_AT_LOCALITY_DECISION'] < agetoselect]

## One-hot enconde Locality\_Decision


In [ ]:
def encode_locality_decision(df_):
    """ Encodes the Locality Decision column into three columns """

    df = df_.copy()
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('No Further Action ( Reason Required )' , "No Further Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS Group Activity', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS Youth', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS Enquiry Visit', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS SEND Short breaks/Summer Playschemes.', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS Wellbeing Practitioners', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS Signpost to External Service', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('CFWS 0-5 Pathway', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('Contact and Referral', "Some Action")
    df['LOCALITY_DECISION'] = df['LOCALITY_DECISION'].replace('Contact & Referral', "Some Action")

    SOME_ACTION = [1 if i == "Some Action" else 0 for i in df['LOCALITY_DECISION']]
    NO_ACTION = [1 if i == 'No Further Action' else 0 for i in df['LOCALITY_DECISION']]
    EH_ASSESSMENT = [1 if i== 'EH - Assessment' else 0 for i in df['LOCALITY_DECISION']]

    df['SOME_ACTION'] = SOME_ACTION
    df['NO_ACTION'] = NO_ACTION
    df['EH_SUPPORT'] = EH_ASSESSMENT

    df = df.drop(['LOCALITY_DECISION'], axis=1)
    
    return df

In [ ]:
df = encode_locality_decision(df)

## Selecting individuals (rows) with less than 30% of missing values. 



In [ ]:
percentage = 30
row_miss, arr_ = [], []
arr2_ = []

def select_rows_missing_val(data, percent = percentage):

  # Iterate through the data frame row by row
  for index, row in data.iterrows():
      no_missing_vals = row.isnull().sum()
      row_miss.append(no_missing_vals)
      
      # Selecting data with less than % percent missing values and appending to an array
      if 100*no_missing_vals/data.shape[1] < percent:
          arr_.append(row)
      else:
          arr2_.append(row)
  
  # Selecting data with less than {} missing values
  new_data = pd.DataFrame(arr_)
  new_data2 = pd.DataFrame(arr2_)

  row_miss.clear()
  arr_.clear()
  arr2_.clear()
  return new_data, new_data2

df1, df2 = select_rows_missing_val(df, percentage)

**Descriptive Statistics in the datase (children with less than 30% of missing values**)

In [ ]:
df1_temp = df1

In [ ]:
df1_temp = df1_temp.replace(-1, np.nan)

In [ ]:
# Cleaning
df1_temp.columns = df1_temp.columns.str.replace("_", " ")
df1_temp.columns = df1_temp.columns.str.replace("BIN", "")
df1_temp.columns = df1_temp.columns.str.replace("PCT", "")
df1_temp.columns = df1_temp.columns.str.replace("CNT", "")

In [ ]:
df1_temp.describe()

In [ ]:
print("Total percentage of -1s in data {} %".format(round(df1.stack().value_counts()[-1]*100/(df1.shape[0]* df1.shape[1]),2)))
print("Total percentage of missing values in data {} %".format(round(df1.isnull().sum().sum()*100/(df1.shape[0]* df1.shape[1]),2)))

## Missing values imputation and '-1' ('not applicable') values



In [ ]:
# Impute missing values and make new data into a dataframe

new_df1 = df1.fillna(0)
new_df2 = df2.fillna(0)

In [ ]:
# Deal with '-1's
def one_hot_encode_numbered(df):

    df_ =  df.copy()
    numbered_cols = [i for i in df_.columns if not i.endswith('BIN')]
    
    for i in numbered_cols:    
        mnar = [1 if i == -1 else 0 for i in df_[i]]
        df_["{}_MNAR".format(i)] = mnar

        # Change the values of all the '-1's to 0

        df_[i] = df[i].replace(-1,0)
    
    return df_

   
def one_hot_encode_binary(df):

    df_ = df.copy()
    binary_cols = [i for i in df_.columns if i.endswith('BIN')]
    for i in binary_cols:
    
        on = [1 if j==1 else 0 for j in df_[i]]
        off = [1 if j==0 else 0 for j in df_[i]]
        empty = [1 if j==-1 else 0 for j in df_[i]]

        df_["{}_ON".format(i)] = on
        df_["{}_OFF".format(i)] = off
        df_["{}_MNAR".format(i)] = empty
        df_ = df_.drop(i, axis=1)

    return df_ 



new_df1 = one_hot_encode_numbered(new_df1)
new_df1 = one_hot_encode_binary(new_df1)

new_df2 = one_hot_encode_numbered(new_df2)
new_df2 = one_hot_encode_binary(new_df2)


In [ ]:
# Dropping unnecessary columns created in the one-hot encode.
columnlist2 = ['UID_MNAR', 'AGE_AT_LOCALITY_DECISION_MNAR', 'LOCALITY_DECISION_INCIDENCE_MNAR', 
               'MALE_MNAR', 'FEMALE_MNAR', 'OTHER_MNAR', 'SOME_ACTION_MNAR', 'NO_ACTION_MNAR', 'EH_SUPPORT_MNAR']

new_df1 = drop_columns(new_df1, columnlist2)
new_df2 = drop_columns(new_df2, columnlist2)

In [ ]:
# The preprocessed and imputed dataset with individuals who are under 18 years old
new_df = new_df1.append(new_df2)

Pre-processed and imputed dataset is exported to be used for machine learning tasks. 


In [ ]:
path1 = '/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper/PreprocMissingValuesLessThan.csv'
with open(path1, 'w', encoding = 'utf-8-sig') as f1:
  new_df1.to_csv(f1)

## Demographic features

In [ ]:
# Loading the preprocessed dataset and the demographic features
df_final = pd.read_csv('/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper/PreprocMissingValuesLessThanDemographic.csv')
df_final.head()

In [ ]:
df_final = df_final.drop(['Unnamed: 0'], axis = 1)
df_final.head()

# Machine learning models, fairness assessment and bias mitigation 

In [ ]:
# Installing some packages
!pip install --upgrade fairlearn==0.7.0
!pip install --upgrade scikit-learn
!pip install --upgrade seaborn


In [ ]:
from fairlearn.metrics import (
    MetricFrame,
    true_positive_rate,
    false_positive_rate,
    false_negative_rate,
    selection_rate,
    count,
    false_negative_rate_difference
)

from fairlearn.postprocessing import ThresholdOptimizer, plot_threshold_optimizer
from fairlearn.postprocessing._interpolated_thresholder import InterpolatedThresholder
from fairlearn.postprocessing._threshold_operation import ThresholdOperation
from fairlearn.reductions import ExponentiatedGradient, EqualizedOdds, TruePositiveRateParity

In [ ]:
from IPython import display
from datetime import date

### EH_SUPPORT ML model

In [ ]:
df_final = df_final.drop(['UID', 'PBR_ID'], axis=1)


In [ ]:
# This is for the fairnness and bias mitigation purpose
df_final['GENDER'] = (df_final['FEMALE']+2*df_final['MALE']+3*df_final['OTHER'])-1

map_dict = {0: 'FEMALE', 1: 'MALE', 2: 'OTHER'}
df_final['GENDER'] = df_final['GENDER'].map(map_dict)

In [ ]:
# Removing NaN values in the IDACI feature
df_final[['IDACI_SCORE_CLASS']] = df_final[['IDACI_SCORE_CLASS']].fillna(df_final.IDACI_SCORE_CLASS.sample(1).values[0])

In [ ]:
df_final.rename(columns={'IDACI_SCORE_CLASS':'IDACI CLASS'}, inplace=True)

In [ ]:
mapping = {
    'IDACI_1': 'IDACI 1',
    'IDACI_2': 'IDACI 2',
    'IDACI_3': 'IDACI 3',
    'IDACI_4': 'IDACI 4',
    'IDACI_5': 'IDACI 5', }
df_final['IDACI CLASS'] = df_final['IDACI CLASS'].replace(mapping, regex=True)

#### Training the model

In [ ]:
# Defining GENDER as sensitive feature
target_variable = "EH_SUPPORT"
sensitive = ["GENDER"]
demographic = ["GENDER"]

# Defining IDACI as sensitive feature
#target_variable = "EH_SUPPORT"
#demographic = ["IDACI CLASS"]
#sensitive = ["IDACI CLASS"]


In [ ]:
Y, A = df_final.loc[:, target_variable], df_final.loc[:, sensitive]

In [ ]:
# Prepare training and test sets

from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    accuracy_score,
    recall_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve
    )

remove_listcolumns = ['EH_SUPPORT', 'SOME_ACTION', 'NO_ACTION', 'MALE', 'FEMALE', 'OTHER', 'GENDER', 
                      'ETHNICITY', 'FIRST_LANGUAGE', 'IMD_SCORE', 'IDACI_SCORE', 'IMD_SCORE_CLASS', 'IDACI CLASS', 'IMD_SCORE_missing', 
                      'IDACI_SCORE_missing', 'WHITE', 'ASIAN', 'MIXED', 'OTHERETHNIC', 'BLACK', 'NOTSTATED', 
                      'ETHNICITY_missing', 'ENGLISH', 'OTHERLANGUAGE', 'NOTSTATED_LANGUAGE', 'LANGUAGE_missing']
X = df_final.drop(remove_listcolumns, axis=1)
y = df_final[["EH_SUPPORT"]]

# Split data into train and test sets
X_train, X_test, Y_train, Y_test, A_train, A_test, df_final_train, df_final_test = train_test_split(
    X,
    Y,
    A,
    df_final,
    test_size=0.30,
    stratify=y,
    random_state=123
)

In [ ]:
# Fit the ML model

from sklearn.linear_model import LogisticRegression

LR = LogisticRegression(max_iter= 1000, random_state=123).fit(X_train, Y_train)

y_pred = LR.predict(X_test)

THRESHOLD = 0.25
y_pred_thres = np.where(LR.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

Now, let's check the model performance on test set.

In [ ]:
# ROC curve, AUC and optimal ROC point

def plot_opt_cutoff_roc(y_true, predicted_prob):

  import numpy as np
  import matplotlib.pyplot as plt
  from sklearn.metrics import roc_curve, auc

  # Generate some sample data
  y_true = y_true
  y_scores = predicted_prob

  # Calculate the fpr and tpr for all thresholds of the classification
  fpr, tpr, thresholds = roc_curve(y_true, y_scores)

  # Find the optimal threshold
  optimal_idx = np.argmax(tpr - fpr)
  optimal_threshold = thresholds[optimal_idx]

  # Plot the ROC curve
  plt.plot(fpr, tpr, color='darkorange', label='ROC curve (AUC = %0.2f)' % auc(fpr, tpr))
  plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel('False Positive Rate')
  plt.ylabel('True Positive Rate')
  plt.title('Receiver operating characteristic')
  plt.legend(loc="lower right")

  # Plot the optimal point on the ROC curve
  plt.scatter(fpr[optimal_idx], tpr[optimal_idx], marker='o', color='red', label='Optimal point (threshold = %0.2f)' % optimal_threshold)
  plt.show()

In [ ]:
# ROC curve and AUC
plot_opt_cutoff_roc(Y_test, LR.predict_proba(X_test)[:,1])

In [ ]:
# Evaluation metrics
def evaluate_model_performance(y_test, y_pred):
    from sklearn.metrics import classification_report, confusion_matrix
    print('\n Confusion Matrix \n')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

evaluate_model_performance(Y_test, y_pred_thres)

#### Fairness assessment

In [ ]:
# GENDER and IDACI

metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred_thres,
                  sensitive_features=df_final_test['GENDER'])
                  #sensitive_features=df_final_test['IDACI CLASS'])

# The disaggregated metrics are then stored in a pandas DataFrame:

metricframe_unmitigated.by_group

#### Bias mitigation algorithms

#### `ThresholdOptimizer` algorithm

In [ ]:
# ThresholdOptimizer with the logistic regression estimator
TO_est = ThresholdOptimizer(
    estimator=LR,
    constraints="false_negative_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba'
)

In [ ]:
TO_est.fit(X_train, Y_train, sensitive_features=A_train)

ThresholdOptimizer(constraints='false_negative_rate_parity',
                   estimator=LogisticRegression(max_iter=1000,
                                                random_state=123),
                   objective='balanced_accuracy_score',
                   predict_method='predict_proba', prefit=True)

In [ ]:
# Record and evaluate the output of the trained ThresholdOptimizer on test data

Y_pred_postprocess = TO_est.predict(X_test, sensitive_features=A_test, random_state=123)
metricframe_postprocess = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_postprocess,
    sensitive_features=A_test
)

In [ ]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_postprocess.by_group],
           keys=['Unmitigated', 'ThresholdOptimizer'],
           axis=1)

#### Reductions approach with `ExponentiatedGradient`

In [ ]:
expgrad_est = ExponentiatedGradient(
    estimator=LogisticRegression(max_iter=1000, random_state=123),
    constraints=TruePositiveRateParity(difference_bound=0.02)
)

In [ ]:
# Fit the exponentiated gradient model
expgrad_est.fit(X_train, Y_train, sensitive_features=A_train)

In [ ]:
# Record and evaluate predictions on test data

Y_pred_reductions = expgrad_est.predict(X_test, random_state=123)
metricframe_reductions = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_reductions,
    sensitive_features=A_test
)
metricframe_reductions.by_group

#### Comparing performance of the different bias mitigation techniques

In [ ]:
def plot_technique_comparison(mf_dict, metric):
  """
  Plots a specified metric for a given dictionary of MetricFrames.
  """
  mf_dict = {k:v.by_group[metric] for (k,v) in mf_dict.items()}
  comparison_df = pd.DataFrame.from_dict(mf_dict)
  comparison_df.plot.bar(figsize=(14, 8), legend=False)
  plt.grid(False)
  plt.ylabel("False Negative Rate")
  plt.xticks(rotation=-12, ha='center');
  plt.legend(bbox_to_anchor=(0., 1.02, 1., .102), loc='lower left',
                      ncols=3, mode="expand", borderaxespad=0.)

In [ ]:
test_dict = {
    "Unmitigated": metricframe_unmitigated,
    "Post-processing": metricframe_postprocess,
    "Reductions": metricframe_reductions
 }

In [ ]:
# Plot the false negative rate

plot_technique_comparison(test_dict, "false_negative_rate")
plt.show()

In [ ]:
# Overall performance
overall_df = pd.DataFrame.from_dict({
    "Unmitigated": metricframe_unmitigated.overall,
    "Post-processing": metricframe_postprocess.overall,
    "Reductions": metricframe_reductions.overall
})

In [ ]:
overall_df.T

#### LIME results

In [ ]:
#Load the dataset
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper/PreprocMissingValuesLessThan.csv')
df = df.drop(['Unnamed: 0','UID'], axis=1)

In [ ]:
!pip install lime

In [ ]:
#import packages
from __future__ import print_function
import sklearn
import sklearn.datasets
import sklearn.ensemble
import numpy as np
import matplotlib.pyplot as plt
import lime
import lime.lime_tabular
import random
import pandas as pd
pd.set_option("display.float_format", "{:.3f}".format)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(["EH_SUPPORT", "SOME_ACTION", "NO_ACTION"], axis=1)
y = df[["EH_SUPPORT"]]

# Our target variable
target_variable = "EH_SUPPORT"
Y = df.loc[:, target_variable]

# Obtain the labels for the later use by the plot of LIME
columnsName = X.columns
feature_names = [str(columnsName[i]) for i in range(len(columnsName))]  # feature labels
target_names = np.array(['Not EH_SUPPORT', 'EH_SUPPORT']) # target labels

# Convert the data to ndarry
X = np.array(X)
Y = np.array(Y)

# Split the data as train and test sets
train, test, labels_train, labels_test = train_test_split(X, Y, train_size=0.70, stratify = Y, random_state=123)

# Fit the model
LR = LogisticRegression(random_state=123, max_iter=1000).fit(train, labels_train)

THRESHOLD = 0.25
class_prob = np.where(LR.predict_proba(test)[:,1] > THRESHOLD, 1, 0)



##### Most important features: true negative group

In [ ]:
# Load the explainer
LIMEexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=columnsName, class_names=target_names, discretize_continuous=True, mode="classification",
                                                   verbose=False, random_state=123)

# Explaining an instance
exp = LIMEexplainer.explain_instance(test[0], LR.predict_proba, num_features=20)

In [ ]:
# Most important features for the TN group

dt2 = exp.as_list()
dt2 = pd.DataFrame(dt2)
dt2.columns = ['Features', 'LIME values']
dt2 = dt2[dt2['LIME values'] < 0] 

In [ ]:
def plot_lime_bestfeatures(dt):
  """
  Plots a lime barplot with the lime values for the best features.
  """
  import matplotlib.pyplot as plt
  import seaborn as sns
  plt.rcParams['font.size'] = '13'
  
  dtf = pd.DataFrame(dt)

  # cleaning
  dtf = dtf.replace("_", " ", regex=True)
  dtf = dtf.replace("BIN ON", "", regex=True)
  dtf = dtf.replace("CNT", "", regex=True)
  dtf = dtf.replace("BIN OFF", "", regex=True)
  dtf = dtf.replace("BIN MNAR", "NA", regex=True)
  dtf = dtf.replace("PCT", "", regex=True)
  dtf = dtf.replace("MNAR", "NA", regex=True)
  dtf = dtf.replace("<= 0.00", " ", regex=True)
  dtf = dtf.replace("0.00 < ", " ", regex=True)
  dtf = dtf.replace("<= 1.00", " ", regex=True)
  
  dtf.columns = ['Features', 'LIME values']
  
  plt.bar(x = 'Features', height = 'LIME values', data = dtf)
  plt.xticks(rotation=90)
  plt.xlabel("Features")
  plt.ylabel("LIME values")
  plt.grid(False)
  plt.tight_layout()
  plt.legend('')
  
  plt.show()

In [ ]:
plot_lime_bestfeatures(dt2)

##### Most important features: true positive group

In [ ]:
#Load the explainer
LIMEexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=columnsName, class_names=target_names, discretize_continuous=True, mode="classification",
                                                   verbose=False, random_state=123)

#Explaining an instance
exp = LIMEexplainer.explain_instance(test[1], LR.predict_proba, num_features=20)

In [ ]:
# Most important features for the TP group
dt2 = exp.as_list()
dt2 = pd.DataFrame(dt2)
dt2.columns = ['Features', 'LIME values']
dt2 = dt2[dt2['LIME values'] > 0] 

In [ ]:
def plot_lime_bestfeatures(dt):
  """
  Plots a lime barplot with the lime values for the best features.
  """
  import matplotlib.pyplot as plt
  #import seaborn as sns
  plt.rcParams['font.size'] = '13'

  dtf = pd.DataFrame(dt)

  # cleaning
  dtf = dtf.replace("_", " ", regex=True)
  dtf = dtf.replace("BIN ON", "", regex=True)
  dtf = dtf.replace("CNT", "", regex=True)
  dtf = dtf.replace("BIN OFF", "", regex=True)
  dtf = dtf.replace("BIN MNAR", "NA", regex=True)
  dtf = dtf.replace("PCT", "", regex=True)
  dtf = dtf.replace("MNAR", "NA", regex=True)
  dtf = dtf.replace("<= 0.00", " ", regex=True)
  dtf = dtf.replace("0.00 < ", " ", regex=True)
  dtf = dtf.replace("<= 1.00", " ", regex=True)
  
  dtf.columns = ['Features', 'LIME values']
  
  plt.bar(x = 'Features', height = 'LIME values', data = dtf)
  plt.xticks(rotation=90)
  plt.xlabel("Features")
  plt.ylabel("LIME values")
  plt.grid(False)
  plt.tight_layout()
  plt.legend('')
  
  plt.show()

In [ ]:
plot_lime_bestfeatures(dt2)

## SOME_ACTION



In [ ]:
# Create CLASS_AGE and ATTENDANCE BIN
df_final['CLASS_AGE'] = pd.cut(df_final['AGE_AT_LOCALITY_DECISION'], bins=[0,7.5,12.5,17.5], include_lowest = True, labels=['below 7.5','between 7.5-12.5','above 12.5'])
df_final['ATTENDANCE_YEAR4_BIN'] = pd.cut(df_final['ATTENDANCE_YEAR4_PCT'], bins=[0,0.5,1], include_lowest = True, labels=['<= 0.5', ' > 0.5'])


### Training the model

In [ ]:
# Defining the sensitive features

target_variable = "SOME_ACTION"
demographic = ["CLASS_AGE", "ATTENDANCE_YEAR4_BIN"]
sensitive = ["CLASS_AGE", "ATTENDANCE_YEAR4_BIN"]

In [ ]:
Y, A = df_final.loc[:, target_variable], df_final.loc[:, sensitive]

In [ ]:
# Prepare training and test sets
from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    accuracy_score,
    recall_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve
    )

remove_listcolumns = ['EH_SUPPORT', 'SOME_ACTION', 'NO_ACTION', 'GENDER', 'CLASS_AGE', 'ATTENDANCE_YEAR4_BIN', 
                      'ETHNICITY', 'FIRST_LANGUAGE', 'IMD_SCORE', 'IDACI_SCORE', 'IMD_SCORE_CLASS', 'IDACI CLASS', 'IMD_SCORE_missing', 
                      'IDACI_SCORE_missing', 'WHITE', 'ASIAN', 'MIXED', 'OTHERETHNIC', 'BLACK', 'NOTSTATED', 
                      'ETHNICITY_missing', 'ENGLISH', 'OTHERLANGUAGE', 'NOTSTATED_LANGUAGE', 'LANGUAGE_missing']
X = df_final.drop(remove_listcolumns, axis=1)
y = df_final[["SOME_ACTION"]]

# Split data into train and test sets
X_train, X_test, Y_train, Y_test, A_train, A_test, df_final_train, df_final_test = train_test_split(
    X,
    Y,
    A,
    df_final,
    test_size=0.30,
    stratify=Y,
    random_state=123
)

In [ ]:
# Fit the ML model

from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=123).fit(X_train, Y_train)

y_pred = GBC.predict(X_test)

Now, let's check the model performance on test set.

In [ ]:
# ROC curve and AUC
plot_opt_cutoff_roc(Y_test, GBC.predict_proba(X_test)[:,1])

In [ ]:
# Evaluation metrics

def evaluate_model_performance(y_test, y_pred):
    from sklearn.metrics import classification_report, confusion_matrix
    print('\n Confusion Matrix \n')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

evaluate_model_performance(Y_test, y_pred)

### Fairness assessment

In [ ]:
metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred,
                  sensitive_features=df_final_test['CLASS_AGE'])

# The disaggregated metrics are then stored in a pandas DataFrame:

metricframe_unmitigated.by_group

In [ ]:
metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred,
                  sensitive_features=df_final_test['ATTENDANCE_YEAR4_BIN'])

# The disaggregated metrics are then stored in a pandas DataFrame:

metricframe_unmitigated.by_group

### Bias mitigation algorithms

### `ThresholdOptimizer` algorithm

In [ ]:
# ThresholdOptimizer with the GBC estimator
TO_est = ThresholdOptimizer(
    estimator=GBC,
    constraints="false_negative_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba'
)

In [ ]:
TO_est.fit(X_train, Y_train, sensitive_features=A_train)

ThresholdOptimizer(constraints='false_negative_rate_parity',
                   estimator=GradientBoostingClassifier(random_state=123),
                   objective='balanced_accuracy_score',
                   predict_method='predict_proba', prefit=True)

In [ ]:
metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred,
                  sensitive_features=df_final_test[['CLASS_AGE', 'ATTENDANCE_YEAR4_BIN']])

# Record and evaluate the output of the trained ThresholdOptimizer on test data

Y_pred_postprocess = TO_est.predict(X_test, sensitive_features=A_test, random_state=123)
metricframe_postprocess = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_postprocess,
    sensitive_features=A_test
)

### Reductions approach with `ExponentiatedGradient`

In [ ]:
expgrad_est = ExponentiatedGradient(
    estimator=GradientBoostingClassifier(random_state=123),
    constraints=TruePositiveRateParity(difference_bound=0.02)
)

In [ ]:
# Fit the exponentiated gradient model
expgrad_est.fit(X_train, Y_train, sensitive_features=A_train)

In [ ]:
# Record and evaluate predictions on test data

metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred,
                  sensitive_features=df_final_test[['CLASS_AGE', 'ATTENDANCE_YEAR4_BIN']])

Y_pred_reductions = expgrad_est.predict(X_test, random_state=123)
metricframe_reductions = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_reductions,
    sensitive_features=A_test
)

In [ ]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_reductions.by_group],
           keys=['Unmitigated', 'ExponentiatedGradient'],
           axis=1)

### Predictive performance of the GBC fairness-unaware model

In [ ]:
# Evaluation metrics

def evaluate_model_performance(y_test, y_pred):
    from sklearn.metrics import classification_report, confusion_matrix
    print('\n Confusion Matrix \n')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

evaluate_model_performance(Y_test, Y_pred_reductions)


### Comparing performance of the different techniques

In [ ]:
test_dict = {
    "Unmitigated": metricframe_unmitigated,
    "Post-processing": metricframe_postprocess,
    "Reductions": metricframe_reductions
 }

In [ ]:
plot_technique_comparison(test_dict, "false_negative_rate")
plt.show()

In [ ]:
# Overall performance

overall_df = pd.DataFrame.from_dict({
    "Unmitigated": metricframe_unmitigated.overall,
    "Post-processing": metricframe_postprocess.overall,
    "Reductions": metricframe_reductions.overall
})

In [ ]:
overall_df.T

### LIME results

In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper/PreprocMissingValuesLessThan.csv')
df = df.drop(['Unnamed: 0','UID'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(["EH_SUPPORT", "SOME_ACTION", "NO_ACTION"], axis=1)
y = df[["SOME_ACTION"]]

# Our target variable
target_variable = "SOME_ACTION"
Y = df.loc[:, target_variable]

# Obtain the labels for the later use by the plot of LIME
columnsName = X.columns
feature_names = [str(columnsName[i]) for i in range(len(columnsName))]  # feature labels
target_names = np.array(['Not SOME_ACTION', 'SOME_ACTION']) # target labels

# Convert the data to ndarry
X = np.array(X)
Y = np.array(Y)

# Split the data as train and test sets
train, test, labels_train, labels_test = train_test_split(X, Y, train_size=0.70, stratify = Y, random_state=123)

# Fit the model
GBC = GradientBoostingClassifier(random_state=123).fit(train, labels_train)

y_pred = GBC.predict(test)



##### Most important features: true negative group

In [ ]:
#Load the explainer
LIMEexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=columnsName, class_names=target_names, discretize_continuous=True, mode="classification",
                                                   verbose=False, random_state=123)

#Explaining an instance
exp = LIMEexplainer.explain_instance(test[14], GBC.predict_proba, num_features=20)

In [ ]:
# Most important features for the TN group
dt2 = exp.as_list()
dt2 = pd.DataFrame(dt2)
dt2.columns = ['Features', 'LIME values']
dt2 = dt2[dt2['LIME values'] < 0] 

In [ ]:
def plot_lime_bestfeatures(dt):
  """
  Plots a lime barplot with the lime values for the best features.
  """
  import matplotlib.pyplot as plt
  import seaborn as sns
  plt.rcParams['font.size'] = '13'
  
  dtf = pd.DataFrame(dt)

  # cleaning
  dtf = dtf.replace("_", " ", regex=True)
  dtf = dtf.replace("BIN ON", "", regex=True)
  dtf = dtf.replace("CNT", "", regex=True)
  dtf = dtf.replace("BIN OFF", "", regex=True)
  dtf = dtf.replace("BIN MNAR", "NA", regex=True)
  dtf = dtf.replace("PCT", "", regex=True)
  dtf = dtf.replace("MNAR", "NA", regex=True)
  dtf = dtf.replace("<= 0.00", " ", regex=True)
  dtf = dtf.replace("> 0.00", " ", regex=True)
  dtf = dtf.replace("0.00 < ", " ", regex=True)
  dtf = dtf.replace("<= 1.00", " ", regex=True)
  
  dtf.columns = ['Features', 'LIME values']
  
  plt.bar(x = 'Features', height = 'LIME values', data = dtf)
  plt.xticks(rotation=90)
  plt.xlabel("Features")
  plt.ylabel("LIME values")
  plt.grid(False)
  plt.tight_layout()
  plt.legend('')

  plt.show()

In [ ]:
plot_lime_bestfeatures(dt2)

##### Most important features: true positive group

In [ ]:
#Load the explainer
LIMEexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=columnsName, class_names=target_names, discretize_continuous=True, mode="classification",
                                                   verbose=False, random_state=123)

#Explaining an instance
exp = LIMEexplainer.explain_instance(test[0], GBC.predict_proba, num_features=20)

In [ ]:
# Most important features for the TP group
dt2 = exp.as_list()
dt2 = pd.DataFrame(dt2)
dt2.columns = ['Features', 'LIME values']
dt2 = dt2[dt2['LIME values'] > 0] 

In [ ]:
def plot_lime_bestfeatures(dt):
  """
  Plots a lime barplot with the lime values for the best features.
  """
  import matplotlib.pyplot as plt
  import seaborn as sns
  plt.rcParams['font.size'] = '13'
  
  dtf = pd.DataFrame(dt)

  # cleaning
  dtf = dtf.replace("_", " ", regex=True)
  dtf = dtf.replace("BIN ON", "", regex=True)
  dtf = dtf.replace("CNT", "", regex=True)
  dtf = dtf.replace("BIN OFF", "", regex=True)
  dtf = dtf.replace("BIN MNAR", "NA", regex=True)
  dtf = dtf.replace("PCT", "", regex=True)
  dtf = dtf.replace("MNAR", "NA", regex=True)
  dtf = dtf.replace("<= 0.00", " ", regex=True)
  dtf = dtf.replace("> 0.00", " ", regex=True)
  dtf = dtf.replace("<= 1.00", " ", regex=True)
  
  dtf.columns = ['Features', 'LIME values']
  
  plt.bar(x = 'Features', height = 'LIME values', data = dtf)
  plt.xticks(rotation=90)
  plt.xlabel("Features")
  plt.ylabel("LIME values")
  plt.grid(False)
  plt.tight_layout()
  plt.legend('')

  plt.show()

In [ ]:
plot_lime_bestfeatures(dt2)

## NO_ACTION



### Training the model

In [ ]:
# Defining the sensitive feature

target_variable = "NO_ACTION"
demographic = ["CLASS_AGE"]
sensitive = ["CLASS_AGE"]

In [ ]:
Y, A = df_final.loc[:, target_variable], df_final.loc[:, sensitive]

In [ ]:
# Prepare training and test sets

from sklearn.model_selection import train_test_split

from sklearn.metrics import (
    balanced_accuracy_score,
    roc_auc_score,
    accuracy_score,
    recall_score,
    confusion_matrix,
    roc_auc_score,
    roc_curve
    )

remove_listcolumns = ['EH_SUPPORT', 'SOME_ACTION', 'NO_ACTION', 'GENDER', 'CLASS_AGE', 'ATTENDANCE_YEAR4_BIN', 
                      'ETHNICITY', 'FIRST_LANGUAGE', 'IMD_SCORE', 'IDACI_SCORE', 'IMD_SCORE_CLASS', 'IDACI CLASS', 'IMD_SCORE_missing', 
                      'IDACI_SCORE_missing', 'WHITE', 'ASIAN', 'MIXED', 'OTHERETHNIC', 'BLACK', 'NOTSTATED', 
                      'ETHNICITY_missing', 'ENGLISH', 'OTHERLANGUAGE', 'NOTSTATED_LANGUAGE', 'LANGUAGE_missing']
X = df_final.drop(remove_listcolumns, axis=1)
y = df_final[["NO_ACTION"]]

# Split data into train and test sets
X_train, X_test, Y_train, Y_test, A_train, A_test, df_final_train, df_final_test = train_test_split(
    X,
    Y,
    A,
    df_final,
    test_size=0.30,
    stratify=Y,
    random_state=123
)

In [ ]:
# Fit the ML model

from sklearn.linear_model import LogisticRegression
lr2 = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', random_state=123).fit(X_train, Y_train)

y_pred = lr2.predict(X_test)

THRESHOLD = 0.46
y_pred_thres = np.where(lr2.predict_proba(X_test)[:,1] > THRESHOLD, 1, 0)

Now, let's check the model performance on test set.

In [ ]:
# ROC curve and AUC

plot_opt_cutoff_roc(Y_test, lr2.predict_proba(X_test)[:,1])

In [ ]:
# Evaluation metrics

def evaluate_model_performance(y_test, y_pred):
    from sklearn.metrics import classification_report, confusion_matrix
    print('\n Confusion Matrix \n')
    print(confusion_matrix(y_test, y_pred))
    print(classification_report(y_test, y_pred))

evaluate_model_performance(Y_test, y_pred_thres)

### Fairness assessment

In [ ]:
metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred_thres,
                  sensitive_features=df_final_test['CLASS_AGE'])

# The disaggregated metrics are then stored in a pandas DataFrame:

metricframe_unmitigated.by_group

### Bias mitigatio algorithms

### `ThresholdOptimizer` algorithm

In [ ]:
# ThresholdOptimizer with the logistic regression estimator
TO_est = ThresholdOptimizer(
    estimator=lr2,
    constraints="false_negative_rate_parity",
    objective="balanced_accuracy_score",
    prefit=True,
    predict_method='predict_proba'
)

In [ ]:
TO_est.fit(X_train, Y_train, sensitive_features=A_train)

ThresholdOptimizer(constraints='false_negative_rate_parity',
                   estimator=LogisticRegression(class_weight='balanced',
                                                penalty='l1', random_state=123,
                                                solver='liblinear'),
                   objective='balanced_accuracy_score',
                   predict_method='predict_proba', prefit=True)

In [ ]:
metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred_thres,
                  sensitive_features=df_final_test[['CLASS_AGE']])

# Record and evaluate the output of the trained ThresholdOptimizer on test data

Y_pred_postprocess = TO_est.predict(X_test, sensitive_features=A_test, random_state=123)
metricframe_postprocess = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_postprocess,
    sensitive_features=A_test
)

In [ ]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_postprocess.by_group],
           keys=['Unmitigated', 'ThresholdOptimizer'],
           axis=1)

### Reductions approach with `ExponentiatedGradient`

In [ ]:
expgrad_est = ExponentiatedGradient(
    estimator=LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', max_iter=1000, random_state=123),
    constraints=TruePositiveRateParity(difference_bound=0.04)
)

In [ ]:
# Fit the exponentiated gradient model
expgrad_est.fit(X_train, Y_train, sensitive_features=A_train)

In [ ]:
# Record and evaluate predictions on test data

metrics_dict = {
    "selection_rate": selection_rate,
    "false_negative_rate": false_negative_rate,
    "balanced_accuracy": balanced_accuracy_score,
}

metricframe_unmitigated = MetricFrame(metrics=metrics_dict,
                  y_true=Y_test,
                  y_pred=y_pred_thres,
                  sensitive_features=df_final_test[['CLASS_AGE']])

Y_pred_reductions = expgrad_est.predict(X_test, random_state=123)
metricframe_reductions = MetricFrame(
    metrics=metrics_dict,
    y_true=Y_test,
    y_pred=Y_pred_reductions,
    sensitive_features=A_test
)

In [ ]:
pd.concat([metricframe_unmitigated.by_group,
           metricframe_reductions.by_group],
           keys=['Unmitigated', 'ExponentiatedGradient'],
           axis=1)

### Comparing performance of the different techniques

In [ ]:
test_dict = {
    "Unmitigated": metricframe_unmitigated,
    "Post-processing": metricframe_postprocess,
    "Reductions": metricframe_reductions
 }

In [ ]:
plot_technique_comparison(test_dict, "false_negative_rate")
plt.show()

In [ ]:
# Overall performance

overall_df = pd.DataFrame.from_dict({
    "Unmitigated": metricframe_unmitigated.overall,
    "Post-processing": metricframe_postprocess.overall,
    "Reductions": metricframe_reductions.overall
})

In [ ]:
overall_df.T

### LIME results

In [ ]:
# Load the dataset
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/LCC/Themis.AI Reports/GitHub Paper/PreprocMissingValuesLessThan.csv')
df = df.drop(['Unnamed: 0','UID'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(["EH_SUPPORT", "SOME_ACTION", "NO_ACTION"], axis=1)
y = df[["NO_ACTION"]]

# Our target variable
target_variable = "NO_ACTION"
Y = df.loc[:, target_variable]

# Obtain the labels for the later use by the plot of LIME
columnsName = X.columns
feature_names = [str(columnsName[i]) for i in range(len(columnsName))]  # feature labels
target_names = np.array(['Not NO_ACTION', 'NO_ACTION']) # target labels

# Convert the data to ndarry
X = np.array(X)
Y = np.array(Y)

# Split the data as train and test sets
train, test, labels_train, labels_test = train_test_split(X, Y, train_size=0.70, stratify = Y, random_state=123)

# Fit the model
LR = LogisticRegression(penalty='l1', solver='liblinear', class_weight='balanced', random_state=123).fit(train, labels_train)

THRESHOLD = 0.46
class_prob = np.where(LR.predict_proba(test)[:,1] > THRESHOLD, 1, 0)



##### Most important features: true negative group

In [ ]:
#Load the explainer
LIMEexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=columnsName, class_names=target_names, discretize_continuous=True, mode="classification",
                                                   verbose=False, random_state=123)

#Explaining an instance
exp = LIMEexplainer.explain_instance(test[0], LR.predict_proba, num_features=20)

In [ ]:
# Most important features for the TN group
dt2 = exp.as_list()
dt2 = pd.DataFrame(dt2)
dt2.columns = ['Features', 'LIME values']
dt2 = dt2[dt2['LIME values'] < 0]

In [ ]:
def plot_lime_bestfeatures2(dt):
  """
  Plots a lime barplot with the lime values for the best features.
  """
  import matplotlib.pyplot as plt
  import seaborn as sns
  plt.rcParams['font.size'] = '13'
  
  dtf = pd.DataFrame(dt)

  # cleaning
  dtf = dtf.replace("_", " ", regex=True)
  dtf = dtf.replace("BIN ON", "", regex=True)
  dtf = dtf.replace("CNT", "", regex=True)
  dtf = dtf.replace("BIN OFF", "", regex=True)
  dtf = dtf.replace("BIN MNAR", "NA", regex=True)
  dtf = dtf.replace("PCT", "", regex=True)
  dtf = dtf.replace("MNAR", "NA", regex=True)
  dtf = dtf.replace("<= 0.00", " ", regex=True)
  dtf = dtf.replace("> 0.00", " ", regex=True)
  dtf = dtf.replace("0.00 < ", " ", regex=True)
  dtf = dtf.replace("<= 1.00", " ", regex=True)
  
  dtf.columns = ['Features', 'LIME values']
  
  plt.bar(x = 'Features', height = 'LIME values', data = dtf)
  plt.xticks(rotation=90)
  plt.xlabel("Features")
  plt.ylabel("LIME values")
  plt.grid(False)
  plt.tight_layout()
  plt.legend('')

  plt.show()

In [ ]:
plot_lime_bestfeatures(dt2)

##### Most important features: true positive group

In [ ]:
#Load the explainer
LIMEexplainer = lime.lime_tabular.LimeTabularExplainer(train, feature_names=columnsName, class_names=target_names, discretize_continuous=True, mode="classification",
                                                   verbose=False, random_state=123)

#Explaining an instance
exp = LIMEexplainer.explain_instance(test[19], LR.predict_proba, num_features=20)

In [ ]:
# Most important features for the TP group
dt2 = exp.as_list()
dt2 = pd.DataFrame(dt2)
dt2.columns = ['Features', 'LIME values']
dt2 = dt2[dt2['LIME values'] > 0]

In [ ]:
def plot_lime_bestfeatures2(dt):
  """
  Plots a lime barplot with the lime values for the best features.
  """
  import matplotlib.pyplot as plt
  import seaborn as sns
  plt.rcParams['font.size'] = '13'
  
  dtf = pd.DataFrame(dt)

  # cleaning
  dtf = dtf.replace("_", " ", regex=True)
  dtf = dtf.replace("BIN ON", "", regex=True)
  dtf = dtf.replace("CNT", "", regex=True)
  dtf = dtf.replace("BIN OFF", "", regex=True)
  dtf = dtf.replace("BIN MNAR", "NA", regex=True)
  dtf = dtf.replace("PCT", "", regex=True)
  dtf = dtf.replace("MNAR", "NA", regex=True)
  dtf = dtf.replace("<= 0.00", " ", regex=True)
  dtf = dtf.replace("> 0.00", " ", regex=True)
  dtf = dtf.replace("0.00 < ", " ", regex=True)
  dtf = dtf.replace("<= 1.00", " ", regex=True)
  
  dtf.columns = ['Features', 'LIME values']
  
  plt.bar(x = 'Features', height = 'LIME values', data = dtf)
  plt.xticks(rotation=90)
  plt.xlabel("Features")
  plt.ylabel("LIME values")
  plt.grid(False)
  plt.tight_layout()
  plt.legend('')

  plt.show()

In [ ]:
plot_lime_bestfeatures(dt2)